#  Bayesian analysis for fermionic rotational EFT

## I. K. Alnamlah, E. A. Coello Pérez, D. R. Phillips

## Importing Libraries and Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emcee
import corner


from levels import experiment_levels, Slope, E_r_LO_
from plots import setup_rc_params
setup_rc_params(fontsize=11)

## Defining the error model

The following block defines the theory estimate for the energy levels. This is at any order depending on the size of p, which tells the function how many LEC's it should use.

In [2]:
def En(p,A,x,bandhead):
    n=len(p)
    E=p[0]+A*x*(x+1)
    
    if bandhead == '1/2':
        for i,q in enumerate(p[2::2]):
            if i ==0: E=E-A*x*(x+1)
            E=E+q*(x*(x+1))**(i+1) 
                
        for i,q in enumerate(p[1::2]):
            E=E+q*(-1)**(x+1/2)*(x+1/2)*(x*(x+1))**i
            
    if bandhead == '3/2':
        for i,q in enumerate(p[1::2]):
            if i ==0: E=E-A*x*(x+1)
            E=E+q*(x*(x+1))**(i+1) 
                
        for i,q in enumerate(p[2::2]):
            E=E+q*(-1)**(x+3/2)*(x-1/2)*(x+1/2)*(x+3/2)*(x*(x+1))**i
            
    if bandhead == '5/2':
        for i,q in enumerate(p[1:]):
            if i ==0: E=E-A*x*(x+1)
            E=E+q*(x*(x+1))**(i+1) 
            
    if bandhead == '7/2':
        for i,q in enumerate(p[1:]):
            if i ==0: E=E-A*x*(x+1)
            E=E+q*(x*(x+1))**(i+1) 

    return E

The following block defines the theory error. `k+1` is the first term in the theory error `kmax` is the highest term. 

In [3]:
def error_th(c_bar_odd,c_bar_even,y_ref,Q,x,k,kmax,bandhead):
    error=[0*x*Q]
    if bandhead == '1/2': s=1; structure=(x+1/2)
    if bandhead == '3/2': s=3; structure=Q**2*(x-1/2)*(x+1/2)*(x+3/2)
    if bandhead == '5/2': s=5; structure=Q**4*(x+5/2)*(x+3/2)*(x+1/2)*(x-1/2)*(x-3/2)
    if bandhead == '7/2': s=7; structure=Q**6*(x+7/2)*(x+5/2)*(x+3/2)*(x+1/2)*(x-1/2)*(x-3/2)*(x-5/2)

    for q in np.arange(k+1,kmax+1,1):

        if q%2 ==0 :#if k is even
            row=y_ref*c_bar_even*Q*x*(x+1)*(Q**2*(x*(x+1)))**((q-2)/2)
            error.append(row)
        
        else:
            if q >= s:
                row=y_ref*c_bar_odd*structure*(-1)**(x+s/2)*(Q**2*(x*(x+1)))**((q-s)/2)
                error.append(row)
    return np.array(error[1:])    

The following block has the log of the likelihood.

In [4]:
def log_likelihood(p,c_bar_odd,c_bar_even,Q,k,kmax,y_ref,I_odd,E_odd,error_odd_ex,verbose,bandhead):
   
    residual=E_odd-En(p,y_ref,I_odd,bandhead)
    errorth=error_th(c_bar_odd,c_bar_even,y_ref,Q,I_odd,k,kmax,bandhead)
    #print(np.array(errorth).shape)
    sigma_th=errorth.T @ errorth
    
    sigma_ex=np.diag(error_odd_ex**2)
    
    C= sigma_th + sigma_ex

    det=np.linalg.det(C)
    
    if det==0:
        if verbose: print('singular');
        return -np.inf
    if det<0:
        if verbose:
            print('<0');
            print('c_bar=',c_bar_odd);
            print('Q=',Q);
            print('det=',det)

    prefactor=(2*np.pi)**(-len(I_odd)/2)*det**(-1/2)
    
    log_likelihood=np.log(prefactor)-1/2*residual.T @ np.linalg.inv(C) @ residual
    
    return log_likelihood

## Defining the priors

In [5]:
def log_prior_on_LECs(p,c_bar_odd,c_bar_even,Q,sigma_E,y_ref,bandhead):
    sigma=sigma_E
    prefactor=np.log(1/(sigma*np.sqrt(2*np.pi)))
    log_prior=prefactor-1/2*p[0]**2/sigma**2
    
    for i,q in enumerate(p[1:]):
        if bandhead == '3/2': i=i+1
        if bandhead == '5/2': i=i*2+1
        if bandhead == '7/2': i=i*2+1
            
        if i%2 ==0: c=np.sqrt(c_bar_odd**2);
        else: c=np.sqrt(c_bar_even**2)
            
        sigma=c*y_ref*Q**i
        prefactor=np.log(1/(sigma*np.sqrt(2*np.pi)))
        
        if i ==1: q=q-y_ref
            
        log_prior=log_prior+prefactor-1/2*q**2/sigma**2
    return log_prior

In [6]:
from scipy.special import gamma
def f_inverse_chi_squared(z, nu, tau):
    """
    Density for the scaled inverse chi^2 distribution
    Mode should be nu tau^2 / (nu + 2) and mean nu tau^2 / (nu - 2) for n > 2
    """
    prefactor = (nu * tau**2 / 2)**(nu/2) / gamma(nu/2)
    return prefactor / z**(1 + nu/2) * np.exp(-nu * tau**2 / (2 * z))

In [7]:
import scipy.integrate as integrate
import scipy.special as special

result = integrate.quad(lambda x:f_inverse_chi_squared(x**2, 1, 1)*2*x, 0, 1000)

print(result)

(0.9992021155721773, 5.910356001139788e-10)


## Building the posterior

In [8]:
def log_posterior(q,n_om,y_ref,I_odd,E_odd,error_odd_ex,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,verbose=True):
    
    p=q[:-3]
    Q=q[-3]
    c_bar_even=q[-2]
    c_bar_odd=q[-1]
    
    if Q<=0: return -np.inf
    if Q>Q_cutoff: return -np.inf
    if c_bar_even<=0: return -np.inf
    if c_bar_odd<=0: return -np.inf
    if c_bar_odd>c_bar_cutoff_odd: return -np.inf
    if c_bar_even>c_bar_cutoff_even: return -np.inf
    
    if bandhead == '1/2': k=len(p)-1
    if bandhead == '3/2': k=len(p)
    if bandhead == '5/2': k=len(p)*2-2
    if bandhead == '7/2': k=len(p)*2-2
    kmax=k+n_om
    
    log_likelihood1=log_likelihood(p,c_bar_odd,c_bar_even,Q,k,kmax,y_ref,I_odd,E_odd,error_odd_ex,verbose,bandhead)
    log_prior_on_LECs1=log_prior_on_LECs(p,c_bar_odd,c_bar_even,Q,sigma_E,y_ref,bandhead)
    log_prior_on_Q=0
    log_prior_on_c_bar_odd=np.log(f_inverse_chi_squared(c_bar_odd**2, 1,1))+np.log(2*c_bar_odd)
    log_prior_on_c_bar_even=np.log(f_inverse_chi_squared(c_bar_even**2, 1,1))+np.log(2*c_bar_even)
    
    log_posterior= log_likelihood1+log_prior_on_LECs1+log_prior_on_Q+log_prior_on_c_bar_odd+log_prior_on_c_bar_even
    return log_posterior



## Sampling the Posterior

In [9]:
def sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True):


    ndim=nLECs+3 # No. of dimension in parameter space or No. of parametres
    range_of_starting_guesses=range_of_starting_guesses[:nLECs]+range_of_starting_guesses[-3:]
    starting_guesses = range_of_starting_guesses + (2.0*np.random.rand(nwalkers,ndim)-1.0)*1e-3
    
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[nomitted,A,I_odd,E_odd,error_odd_ex,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead])
    old_tau = np.inf
    sampler.run_mcmc(starting_guesses, 10000,progress=True)

    # Compute the autocorrelation time so far
    # Using tol=0 means that we'll always get an estimate even
    # if it isn't trustworthy
    tau = sampler.get_autocorr_time(tol=0)

    # Check convergence
    print(tau * 50 ,np.mean(tau)*50, sampler.iteration)
    print(np.abs(old_tau - tau) / tau)
    converged = np.all(tau * 50 < sampler.iteration)
    converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
    
    while  converged == False and sampler.iteration<80000:
        old_tau = tau
        sampler.run_mcmc(initial_state= None,nsteps= 3000,progress=True)
        tau = sampler.get_autocorr_time(tol=0)
        print(tau * 50 ,np.mean(tau)*50, sampler.iteration)
        print(np.abs(old_tau - tau) / tau)
        converged = np.all(tau * 50 < sampler.iteration)
        converged &= np.all(np.abs(old_tau - tau) / tau < 0.02)
        
    
    tau = sampler.get_autocorr_time(tol=0)
    burnin = int(2 * np.max(tau))
    thin = int(0.5 * np.min(tau))
    flat_samples = sampler.get_chain(discard=burnin, flat=True, thin=thin)

    print("burn-in: {0}".format(burnin))
    print("thin: {0}".format(thin))
    print("flat chain shape: {0}".format(flat_samples.shape))

    samples = sampler.get_chain()
    if plots:
        if bandhead == '1/2':
            labels=[r'$E_K$',r'$A_1$',r'$A$',r'$B_1$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
        elif bandhead == '3/2':
            labels=[r'$E_K$',r'$A$',r'$A_3$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
        elif bandhead == '5/2':
            labels=[r'$E_K$',r'$A$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
        elif bandhead == '7/2':
            labels=[r'$E_K$',r'$A$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
        labels=labels[:nLECs]+labels[-3:]

        fig1, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
        for i in range(ndim):
            ax = axes[i]
            ax.plot(samples[:, :, i], "k", alpha=0.1)
            ax.set_ylabel(labels[i])

        axes[-1].set_xlabel("step number",fontsize=20);
        plt.show()
        plt.close()
        setup_rc_params(fontsize=16)
        fig2 = corner.corner(flat_samples,labels=labels, show_titles=True,use_math_text=True,verbose=True,max_n_ticks=3,
             label_kwargs={'fontsize':18,'rotation':0},title_fmt='.4f',title_kwargs={'fontsize':14},
             quantiles=[],reverse=False,labelpad=0.2,top_ticks=False,fontsize=18);
        plt.show()
        plt.close()
        setup_rc_params(fontsize=11)
        return samples,tau,fig1,fig2
    else:
        return samples,tau


In [10]:
def nuclei_parm(odd):
    nla=0
    nlb=0
    if odd == r'$^{169}$Er': rotor=r'$^{168}$Er';m=0;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.16,22,22;\
        range_of_starting_guesses=[0.0,10,12,-0.007,-0.004,0.05,2,1]  # set the cuttoffs on the hyperparameters
        
    if odd == r'$^{167}$Er': rotor=r'$^{166}$Er';m=1;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=230,0.08,8,16; \
        range_of_starting_guesses=[0.0,8,11,-0.005,-0.008,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{167}$Er_3/2': rotor=r'$^{166}$Er';m=13;bandhead='3/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=573,0.2,22,22; \
        range_of_starting_guesses=[0.0,11,-0.005,-0.008,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{167}$Er_5/2': rotor=r'$^{166}$Er';m=14;bandhead='5/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=445,0.2,22,22; \
        range_of_starting_guesses=[0.0,11,-0.008,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{167}$Er_7/2': rotor=r'$^{166}$Er';m=15;bandhead='7/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=174,0.2,22,22; \
        range_of_starting_guesses=[0.0,11,-0.008,0.05,2,1]  # set the cuttoffs on the hyperparameters

    if odd == r'$^{169}$Tm': rotor=r'$^{168}$Er';m=2;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.16,22,22; \
        range_of_starting_guesses=[0.0,-10,12,0.03,-0.005,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{167}$Tm': rotor=r'$^{166}$Er';m=3;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.16,22,22;\
        range_of_starting_guesses=[0.0,-9,12,0.04,-0.008,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{99}$Tc': rotor=r'$^{100}$Ru';m=4;nla=5;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=160,0.3,2.5,2.5;\
        range_of_starting_guesses=[0.0,60,80,-3,-1,0.2,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{183}$W': rotor=r'$^{182}$W';m=5;nla=2;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.12,15,22;\
        range_of_starting_guesses=[0.0,3,13,-0.03,0.013,0.05,2,1]  # set the cuttoffs on the hyperparameters
        
    if odd == r'$^{235}$U': rotor=r'$^{234}$U';m=6;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.1,40,40;\
        range_of_starting_guesses=[0.0,-2,6,0.002,-0.003,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{235}$U_5/2': rotor=r'$^{234}$U';m=16;bandhead='5/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=180,0.15,22,22;\
        range_of_starting_guesses=[0.0,6,-0.003,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{235}$U_7/2': rotor=r'$^{234}$U';m=17;bandhead='7/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=100,0.1,22,22;\
        range_of_starting_guesses=[0.0,6,-0.003,0.05,2,1]  # set the cuttoffs on the hyperparameters



    if odd == r'$^{239}$Pu': rotor=r'$^{238}$Pu';m=7;bandhead='1/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=22,0.075,4.5,40 ;\
        range_of_starting_guesses=[0.0,-4,6,0.004,-0.002,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{159}$Dy': rotor=r'$^{158}$Dy';m=8;bandhead='3/2';nla=12;\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=61.8125,0.09,10,20;\
        range_of_starting_guesses=[0.0,11,-0.005,-0.005,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{159}$Gd': rotor=r'$^{158}$Gd';m=9;bandhead='3/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=50,0.12,10,20;\
        range_of_starting_guesses=[0.0,12,-0.01,-0.005,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{157}$Gd': rotor=r'$^{156}$Gd';m=10;bandhead='3/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=55.625,0.12,10,20;\
        range_of_starting_guesses=[0.0,12,-0.01,-0.005,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{155}$Gd': rotor=r'$^{154}$Gd';m=11;bandhead='3/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=76.9375,0.12,10,20;\
        range_of_starting_guesses=[0.0,12,-0.01,-0.005,0.05,2,1]  # set the cuttoffs on the hyperparameters
    if odd == r'$^{153}$Gd': rotor=r'$^{152}$Gd';m=12;bandhead='3/2';\
        sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even=50,0.12,10,20;\
        range_of_starting_guesses=[0.0,12,-0.01,-0.005,0.05,2,1]   #set the cuttoffs on the hyperparameters
    if bandhead == '1/2':
        labels=[r'$E_K$',r'$A_1$',r'$A$',r'$B_1$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
    elif bandhead == '3/2':
        labels=[r'$E_K$',r'$A$',r'$A_3$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
    elif bandhead == '5/2':
        labels=[r'$E_K$',r'$A$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']
    elif bandhead == '7/2':
        labels=[r'$E_K$',r'$A$',r'$B$',r'$Q$',r'$\bar{c}_{\rm even}$',r'$\bar{c}_{\rm odd}$']

    return rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels



In [ ]:
nwalkers,n_steps=64,100000

odd=r'$^{167}$Er' # choose the odd nucleus
nLECs=5                 # choose the number of LEC's (This determines what order you want to calculate in your EFT)
nomitted=6              # and the number of omitted error terms

rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
n=len(I_odd)
E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla],error_odd_ex[nlb:n-nla],I_odd[nlb:n-nla]
A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)

print('for ',odd,'N'+str(nLECs-1)+'LO')
print('number of error terms is ',nomitted)
print('number of levels is ',n-nla-nlb)
print('number of removed levels from above is ',nla)
print('number of removed levels from below is ',nlb)

samples,autocorrelation,fig1,fig2=sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True)
#np.save(    'samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+str(Q_cutoff),[samples,autocorrelation])
#fig1.savefig('chains_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+str(Q_cutoff)+'.png')
#fig2.savefig('corner_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+str(Q_cutoff)+'.png')

In [ ]:
nwalkers,n_steps=64,100000
nuclei=[r'$^{169}$Er',r'$^{167}$Er',r'$^{169}$Tm',r'$^{167}$Tm',r'$^{235}$U',r'$^{239}$Pu',r'$^{183}$W',r'$^{99}$Tc',r'$^{159}$Dy',r'$^{157}$Gd',r'$^{155}$Gd']
 

for odd in nuclei: # choose the odd nucleus
    k=4 
    kmax=10
    
    rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
    E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
    n=len(I_odd)
    E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla],error_odd_ex[nlb:n-nla],I_odd[nlb:n-nla]
    A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)
    
    if bandhead =='1/2': nLECs=k+1
    if bandhead =='3/2': nLECs=k
    if odd ==r'$^{159}$Dy':kmax=12
    nomitted=kmax-k
    
    print('for ',odd,r'N$^%1i$'%k+'LO')
    print('number of error terms is ',nomitted)
    print('number of levels is ',n-nla-nlb)
    print('number of removed levels from above is ',nla)
    print('number of removed levels from below is ',nlb)

    samples,autocorrelation,fig1,fig2=sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True)
    np.save(    'samples_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla),[samples,autocorrelation])
    fig1.savefig('chains_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.png')
    fig2.savefig('corner_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.png')

### Removing Levels from above

In [ ]:
nuclei=[r'$^{169}$Er']

nwalkers,n_steps=64,16500 # set the sampler parameters
kmax=10

for nLECs in range(2,6,1):
    k=nLECs-1
    nomitted=kmax-k
    
    
    for odd in  nuclei:

        rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
        E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
        A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)
        n=len(I_odd)

        for nla1 in range(nla,n-4,1):

            E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla1],error_odd_ex[nlb:n-nla1],I_odd[nlb:n-nla1]
            
   
    
            print('for ',odd,r'N$^%1i$'%k+'LO')
            print('number of error terms is ',nomitted)
            print('number of levels is ',n-nla1-nlb)
            print('number of removed levels from above is ',nla1)
            print('number of removed levels from below is ',nlb)

            samples,autocorrelation,fig1,fig2=sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True)
            np.save(    'samples/samples_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1),[samples,autocorrelation])
            fig1.savefig('samples/chains_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.png')
            fig2.savefig('samples/corner_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.png')

In [ ]:
nuclei=[r'$^{239}$Pu']

nwalkers,n_steps=64,16500 # set the sampler parameters

for nLECs in range(5,6,1):
    k=nLECs-1
    for kmax in [6,8,10]:
        nomitted=kmax-k
        
        for odd in  nuclei:

            rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
            E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
            A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)
            n=len(I_odd)

            for nla1 in range(nla,n-4,1):

                E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla1],error_odd_ex[nlb:n-nla1],I_odd[nlb:n-nla1]



                print('for ',odd,r'N$^%1i$'%k+'LO')
                print('number of error terms is ',nomitted)
                print('number of levels is ',n-nla1-nlb)
                print('number of removed levels from above is ',nla1)
                print('number of removed levels from below is ',nlb)

                samples,autocorrelation,fig1,fig2=sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True)
                np.save(    'samples/samples_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1),[samples,autocorrelation])
                fig1.savefig('samples/chains_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.png')
                fig2.savefig('samples/corner_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.png')

### Removing Levels from below

In [ ]:
nuclei=[r'$^{169}$Er']


nwalkers,n_steps=64,21500 # set the sampler parameters
for nLECs in [5]:
    k=nLECs-1
    for kmax in [10]:
        nomitted=kmax-k
        for odd in  nuclei:
            
            rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
            E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
            A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)
            n=len(I_odd)

            for nlb in range(1,n-nla-4,1):

                E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla],error_odd_ex[nlb:n-nla],I_odd[nlb:n-nla]
    
                print('for ',odd,r'N$^%1i$'%k+'LO')
                print('number of error terms is ',nomitted)
                print('number of levels is ',n-nla-nlb)
                print('number of removed levels from above is ',nla)
                print('number of removed levels from below is ',nlb)

                samples,autocorrelation,fig1,fig2=sample(nwalkers,n_steps,range_of_starting_guesses,sigma_E,nLECs,nomitted,A,I_odd,E_odd,error_odd_ex,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,bandhead,plots=True)
                np.save(    'samples/samples_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla),[samples,autocorrelation])
                fig1.savefig('samples/chains_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.png')
                fig2.savefig('samples/corner_'+odd+'_N'+str(k)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.png')

## Saving Parameters

In [ ]:
nuclei=[r'$^{169}$Er',r'$^{167}$Er',r'$^{169}$Tm',r'$^{167}$Tm',r'$^{235}$U',r'$^{239}$Pu',r'$^{183}$W',r'$^{99}$Tc',r'$^{159}$Dy',r'$^{157}$Gd',r'$^{155}$Gd']


for odd in nuclei:
    
    rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
    E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
    n=len(I_odd)
        
    for nLECs in range(5,6,1):
        
        for nomitted in [6]:
            
            if odd == r'$^{159}$Dy': nomitted=8; nla=12
            samples,autocorrelation=np.load('samples/samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.npy',allow_pickle=True)
            tau = autocorrelation
            burnin = int(2 * np.max(tau))
            thin = int(0.5 * np.min(tau))

            flat_samples=samples[burnin::thin, :, :]
            flat_samples = [item for sublist in flat_samples for item in sublist]

            np.save('parameters for plots/flat_samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.npy',flat_samples,allow_pickle=True)

In [ ]:
nuclei=[r'$^{169}$Er',r'$^{167}$Er',r'$^{169}$Tm',r'$^{167}$Tm',r'$^{235}$U',r'$^{239}$Pu',r'$^{183}$W',r'$^{99}$Tc',r'$^{159}$Dy',r'$^{157}$Gd',r'$^{155}$Gd']


for odd in nuclei:

    ll=5

    for LEC in [0,1,2,3,4,-3,-2,-1]:
        print(LEC)

        if LEC >1: s=LEC+1
        else: s=2

        for nLECs in range(s,6,1):


            rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
            E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
            A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)

            n=len(I_odd)
            for nomitted in [4]:
                y11=[]
                for nla1 in range(nla,n-ll,1):
                    E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla1],error_odd_ex[nlb:n-nla1],I_odd[nlb:n-nla1]

                    samples,autocorrelation=np.load('samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.npy',allow_pickle=True)
                    tau = autocorrelation
                    burnin = int(2 * np.max(tau))
                    thin = int(0.5 * np.min(tau))

                    flat_samples=samples[burnin::thin, :, :]
                    flat_samples = [item for sublist in flat_samples for item in sublist]
                    z=np.array(flat_samples).T

                    y=z[LEC]
                    mcmc = np.percentile(y, [16, 50, 84])
                    y1= np.array([mcmc[1],np.diff(mcmc)[0],np.diff(mcmc)[1]])
                    y11.append((y1))

                np.save(str(labels[LEC])+'_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla),y11)



In [ ]:
nuclei=[r'$^{169}$Er',r'$^{167}$Er',r'$^{169}$Tm',r'$^{167}$Tm',r'$^{235}$U',r'$^{239}$Pu',r'$^{183}$W',r'$^{99}$Tc',r'$^{159}$Dy',r'$^{157}$Gd',r'$^{155}$Gd']

ll=5
LEC=2
kmax=8
odd=nuclei[5]
nLECs=5
rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)
nomitted=kmax-nLECs+1


n=len(I_odd)
cor1=[]
for nla1 in range(nla,n-ll,1):
    print(nla1)

    samples,autocorrelation=np.load('samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla1)+'.npy',allow_pickle=True)
    tau = autocorrelation
    burnin = int(2 * np.max(tau))
    thin = int(0.5 * np.min(tau))

    flat_samples=samples[burnin::thin, :, :]
    flat_samples = [item for sublist in flat_samples for item in sublist]
    z=np.array(flat_samples).T
    cor=abs(np.corrcoef(z))
    cor1.append(cor[LEC])
    
np.save('cor_'+str(labels[LEC])+'_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla),cor1)


In [ ]:
nuclei=[r'$^{169}$Er',r'$^{167}$Er',r'$^{169}$Tm',r'$^{167}$Tm',r'$^{235}$U',r'$^{239}$Pu',r'$^{183}$W',r'$^{99}$Tc']
odd=nuclei[0]
ll=4

for LEC in [0]:
    
    if LEC >1: s=LEC+1
    else: s=2
        
    for nLECs in range(5,6,1):

        rotor,bandhead,sigma_E,Q_cutoff,c_bar_cutoff_odd,c_bar_cutoff_even,range_of_starting_guesses,m,nla,nlb,labels=nuclei_parm(odd)
        E_rot,E_vib,E_sp,E_r,error_r_ex,I_r,E_odd,error_odd_ex,I_odd=experiment_levels(rotor,odd)
        A,A_r_NLO,B_r_NLO,E_r=E_r_LO_(E_r,I_r)

        n=len(I_odd)
        for kmax in [10]:
            y11=[]
            for nlb in range(n-nla-6,-1,-1):

                nomitted=kmax-nLECs+1
                E_odd,error_odd_ex,I_odd=E_odd[nlb:n-nla],error_odd_ex[nlb:n-nla],I_odd[nlb:n-nla]
                samples,autocorrelation=np.load('samples/samples_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'.npy',allow_pickle=True)
                tau = autocorrelation
                burnin = int(2 * np.max(tau))
                thin = int(0.5 * np.min(tau))

                flat_samples=samples[burnin::thin, :, :]
                flat_samples = [item for sublist in flat_samples for item in sublist]
                z=np.array(flat_samples).T

                y=z[LEC]
                mcmc = np.percentile(y, [16, 50, 84])
                y1= np.array([mcmc[1],np.diff(mcmc)[0],np.diff(mcmc)[1]])
                y11.append((y1))
                
                if len(z[LEC]) < 50 * np.mean(autocorrelation):
                    print(str(labels[LEC])+'_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'_b')
                    print(len(z[LEC])/autocorrelation[LEC])

            
        np.save('parameters for plots/'+str(labels[LEC])+'_'+odd+'_N'+str(nLECs-1)+'LO_error_'+str(nomitted)+'_levles_'+str(nlb)+'_'+str(n-nla)+'_b',y11)

